### Bài 1:


In [10]:
import sqlite3
import math

# Bước 1: Kết nối tới cơ sở dữ liệu
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# Bước 2: Tạo bảng dữ liệu mẫu nếu chưa có
cursor.execute('''
    CREATE TABLE IF NOT EXISTS data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        a REAL,
        b REAL
    )
''')

# Bước 3: Thêm dữ liệu mẫu (bạn có thể bỏ qua nếu đã có dữ liệu)
sample_data = [(1, 2), (2, 4), (3, 6), (4, 8), (5, 10)]  # Tương quan hoàn hảo
cursor.executemany("INSERT INTO data (a, b) VALUES (?, ?)", sample_data)
conn.commit()

# Bước 4: Lấy các giá trị cần thiết từ SQL
cursor.execute('''
    SELECT 
        COUNT(*) as n,
        SUM(a) as sum_a,
        SUM(b) as sum_b,
        SUM(a * b) as sum_ab,
        SUM(a * a) as sum_a2,
        SUM(b * b) as sum_b2
    FROM data
''')

# Bước 5: Lấy dữ liệu từ kết quả truy vấn
row = cursor.fetchone()
n, sum_a, sum_b, sum_ab, sum_a2, sum_b2 = row

# Bước 6: Tính hệ số tương quan r
numerator = n * sum_ab - sum_a * sum_b
denominator = math.sqrt((n * sum_a2 - sum_a**2) * (n * sum_b2 - sum_b**2))

if denominator == 0:
    print("Không thể tính được hệ số tương quan (mẫu số = 0)")
else:
    r = numerator / denominator
    print(f"Hệ số tương quan Pearson r(A,B) = {r:.4f}")

# Bước 7: Đóng kết nối
conn.close()


Hệ số tương quan Pearson r(A,B) = 1.0000


### Bài 2:

In [ ]:
# câu lệnh sql
#DROP TABLE IF EXISTS car_scores;
#CREATE TABLE car_scores (
#    Day TEXT,
#    Car_Model TEXT,
#    Score REAL
#);

#-- Thêm dữ liệu (long format)
#INSERT INTO car_scores (Day, Car_Model, Score) VALUES
#('Day 1', 'A', 8), ('Day 1', 'B', 9), ('Day 1', 'C', 7),
#('Day 2', 'A', 7.5), ('Day 2', 'B', 8.5), ('Day 2', 'C', 7),
#('Day 3', 'A', 6), ('Day 3', 'B', 7), ('Day 3', 'C', 6),
#('Day 4', 'A', 7), ('Day 4', 'B', 6), ('Day 4', 'C', 5);

#-- Tổng điểm theo mẫu xe
#SELECT Car_Model, SUM(Score) AS Total_Score FROM car_scores GROUP BY Car_Model;

#-- Tổng điểm theo ngày
#SELECT Day, SUM(Score) AS Total_Score FROM car_scores GROUP BY Day;


In [12]:
import sqlite3
import pandas as pd
from scipy.stats import chi2_contingency

# Tạo kết nối SQLite trong bộ nhớ
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng dữ liệu dạng quan hệ
cursor.execute("""
CREATE TABLE car_scores (
    Day TEXT,
    CarModel TEXT,
    Score REAL
)
""")

# Dữ liệu
data = [
    ('Day 1', 'A', 8), ('Day 1', 'B', 9), ('Day 1', 'C', 7),
    ('Day 2', 'A', 7.5), ('Day 2', 'B', 8.5), ('Day 2', 'C', 7),
    ('Day 3', 'A', 6), ('Day 3', 'B', 7), ('Day 3', 'C', 8),
    ('Day 4', 'A', 7), ('Day 4', 'B', 6), ('Day 4', 'C', 5),
]
cursor.executemany("INSERT INTO car_scores (Day, CarModel, Score) VALUES (?, ?, ?)", data)
conn.commit()


In [13]:
# Tạo bảng tạm có phân nhóm điểm (Low / Medium / High)
query_grouped = """
SELECT
    *,
    CASE
        WHEN Score <= 6.5 THEN 'Low'
        WHEN Score <= 8 THEN 'Medium'
        ELSE 'High'
    END AS ScoreGroup
FROM car_scores
"""

df_grouped = pd.read_sql_query(query_grouped, conn)
print(df_grouped)


      Day CarModel  Score ScoreGroup
0   Day 1        A    8.0     Medium
1   Day 1        B    9.0       High
2   Day 1        C    7.0     Medium
3   Day 2        A    7.5     Medium
4   Day 2        B    8.5       High
5   Day 2        C    7.0     Medium
6   Day 3        A    6.0        Low
7   Day 3        B    7.0     Medium
8   Day 3        C    8.0     Medium
9   Day 4        A    7.0     Medium
10  Day 4        B    6.0        Low
11  Day 4        C    5.0        Low


In [14]:
# Bảng chéo giữa CarModel và ScoreGroup
ct_model = pd.crosstab(df_grouped['CarModel'], df_grouped['ScoreGroup'])
print("Bảng tần số theo mẫu xe:\n", ct_model)

# Kiểm định chi bình phương
chi2_model, p_model, dof_model, _ = chi2_contingency(ct_model)
print("\n[Kết quả kiểm định theo mẫu xe]")
print(f"Chi-squared = {chi2_model:.4f}, p-value = {p_model:.4f}")


Bảng tần số theo mẫu xe:
 ScoreGroup  High  Low  Medium
CarModel                     
A              0    1       3
B              2    1       1
C              0    1       3

[Kết quả kiểm định theo mẫu xe]
Chi-squared = 5.1429, p-value = 0.2730


In [15]:
# Bảng chéo giữa Day và ScoreGroup
ct_day = pd.crosstab(df_grouped['Day'], df_grouped['ScoreGroup'])
print("\nBảng tần số theo ngày:\n", ct_day)

# Kiểm định chi bình phương
chi2_day, p_day, dof_day, _ = chi2_contingency(ct_day)
print("\n[Kết quả kiểm định theo ngày]")
print(f"Chi-squared = {chi2_day:.4f}, p-value = {p_day:.4f}")



Bảng tần số theo ngày:
 ScoreGroup  High  Low  Medium
Day                          
Day 1          1    0       2
Day 2          1    0       2
Day 3          0    1       2
Day 4          0    2       1

[Kết quả kiểm định theo ngày]
Chi-squared = 6.0952, p-value = 0.4126


### Bài 3:

In [ ]:
#Câu lệnh sql
#SELECT
#  departure_time,
#  printf('%02d:%02d', departure_time / 100, departure_time % 100) AS formatted_time
#FROM flights;


In [16]:
import sqlite3
import pandas as pd

# Kết nối đến CSDL SQLite
conn = sqlite3.connect('flights.db')
cursor = conn.cursor()

# Tạo bảng flights và thêm dữ liệu mẫu
cursor.execute('''
    CREATE TABLE IF NOT EXISTS flights (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        departure_time INTEGER
    )
''')

# Xóa dữ liệu cũ (nếu có) và thêm mới
cursor.execute("DELETE FROM flights")
sample_times = [(830,), (1445,), (0,), (55,), (123,), (2359,)]
cursor.executemany("INSERT INTO flights (departure_time) VALUES (?)", sample_times)
conn.commit()

# Truy vấn SQL để chuyển departure_time thành định dạng HH:MM
query = '''
SELECT 
    departure_time,
    printf('%02d:%02d', departure_time / 100, departure_time % 100) AS formatted_time
FROM flights
'''

# Đọc dữ liệu thành DataFrame
df = pd.read_sql_query(query, conn)
print(df)


   departure_time formatted_time
0             830          08:30
1            1445          14:45
2               0          00:00
3              55          00:55
4             123          01:23
5            2359          23:59


### Bài 4:


In [ ]:
#Câu lệnh sql
#-- Tính toán Trung vị (Median)
#WITH Median AS (
#    SELECT value
#    FROM data
#    ORDER BY value
#    LIMIT 1 OFFSET (SELECT FLOOR(COUNT(*) / 2) FROM data)
#),

#-- Tính độ lệch tuyệt đối từ Trung vị
#AbsoluteDeviations AS (
#    SELECT
#        value,
#        ABS(value - (SELECT value FROM Median)) AS abs_deviation
#    FROM data
#),

#-- Tính toán MAD (trung vị của các độ lệch tuyệt đối)
#MAD AS (
#    SELECT
#        abs_deviation
#    FROM AbsoluteDeviations
#    ORDER BY abs_deviation
#    LIMIT 1 OFFSET (SELECT FLOOR(COUNT(*) / 2) FROM AbsoluteDeviations)
#)

#-- Lọc các ngoại lệ
#SELECT value
#FROM AbsoluteDeviations
#WHERE abs_deviation > 1.5 * (SELECT abs_deviation FROM MAD);


In [18]:
import sqlite3

# Kết nối đến SQLite
conn = sqlite3.connect("du_lieu.db")

# Tạo bảng dữ liệu nếu chưa tồn tại
conn.execute('''
CREATE TABLE IF NOT EXISTS data_table (
    id INTEGER PRIMARY KEY,
    value REAL
);
''')

# Chèn dữ liệu mẫu vào bảng
conn.execute("INSERT INTO data_table (value) VALUES (10.5), (20.5), (15.5), (100), (2), (50), (3.5)")

# Commit các thay đổi và đóng kết nối
conn.commit()
conn.close()

print("Bảng và dữ liệu mẫu đã được chèn vào cơ sở dữ liệu.")


Bảng và dữ liệu mẫu đã được chèn vào cơ sở dữ liệu.


In [20]:
import sqlite3
import pandas as pd

# Kết nối đến SQLite database
conn = sqlite3.connect("du_lieu.db")

# Đọc dữ liệu từ bảng vào DataFrame
df = pd.read_sql_query("SELECT * FROM data_table", conn)

# Tính median và MAD (Median Absolute Deviation)
median = df['value'].median()  # Tính giá trị trung vị (median) của cột 'value'
mad = (df['value'] - median).abs().median()  # Tính MAD

# Tạo cột độ lệch tuyệt đối
df['abs_dev'] = (df['value'] - median).abs()

# Tìm các giá trị ngoại lệ (các giá trị có độ lệch > 1.5 * MAD)
outliers = df[df['abs_dev'] > 1.5 * mad]

# Hiển thị các giá trị ngoại lệ
print("Các giá trị ngoại lệ:")
print(outliers)

# Đóng kết nối
conn.close()


Các giá trị ngoại lệ:
   id  value  abs_dev
3   4  100.0     84.5
5   6   50.0     34.5


### Bài 5:

In [21]:
import sqlite3
import pandas as pd

# Kết nối tới cơ sở dữ liệu SQLite
conn = sqlite3.connect('du_lieu_2.db')  # Đổi 'du_lieu.db' thành tên cơ sở dữ liệu của bạn

# Tạo bảng và chèn dữ liệu mẫu (nếu chưa có bảng)
conn.execute('''
CREATE TABLE IF NOT EXISTS Patient (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    weight REAL,
    height REAL
);
''')

# Chèn dữ liệu mẫu (bỏ qua nếu đã có dữ liệu trong bảng)
conn.execute("INSERT INTO Patient (last_name, weight, height) VALUES ('Nguyen', 70.0, 175.0)")
conn.execute("INSERT INTO Patient (last_name, weight, height) VALUES ('Nguyen', 71.0, 180.0)")
conn.execute("INSERT INTO Patient (last_name, weight, height) VALUES ('Tran', 70.5, 165.0)")
conn.execute("INSERT INTO Patient (last_name, weight, height) VALUES ('Hoang', 65.0, 160.0)")
conn.commit()

# Truy vấn để so sánh các bệnh nhân
query = """
SELECT 
    p1.last_name AS last_name_1,
    p1.weight AS weight_1,
    p2.last_name AS last_name_2,
    p2.weight AS weight_2,
    CASE 
        WHEN p1.last_name = p2.last_name AND 
             ABS(p1.weight - p2.weight) <= 0.05 * p1.weight THEN 'Possibly the same person'
        ELSE 'Different person'
    END AS comparison_result
FROM 
    Patient p1
JOIN 
    Patient p2
ON 
    p1.id < p2.id;
"""

# Đọc kết quả vào DataFrame
df = pd.read_sql_query(query, conn)

# Hiển thị kết quả
print(df)

# Đóng kết nối
conn.close()


  last_name_1  weight_1 last_name_2  weight_2         comparison_result
0      Nguyen      70.0      Nguyen      71.0  Possibly the same person
1      Nguyen      70.0        Tran      70.5          Different person
2      Nguyen      70.0       Hoang      65.0          Different person
3      Nguyen      71.0        Tran      70.5          Different person
4      Nguyen      71.0       Hoang      65.0          Different person
5        Tran      70.5       Hoang      65.0          Different person
